In [1]:
import tensorflow as tf
import numpy as np
import sklearn
import random,os,io
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Dense
from tensorflow.keras.callbacks import Callback, EarlyStopping, LearningRateScheduler, ModelCheckpoint,CSVLogger
from tensorboard.plugins.hparams import api as hp

In [2]:
import pandas as pd
dep_train=pd.read_csv("D:\\bro\\Analysis\\mSANN\\cd\\XTr.dat",sep='\s+',names=[str(i) for i in range(0,16)])
indep_train=pd.read_csv("D:\\bro\\Analysis\\mSANN\\cd\\yTr.dat",sep='\s+',names=['target'])
dep_val=pd.read_csv("D:\\bro\\Analysis\\mSANN\\cd\\XV.dat",sep='\s+',names=[str(i) for i in range(0,16)])
indep_val=pd.read_csv("D:\\bro\\Analysis\\mSANN\\cd\\yV.dat",sep='\s+',names=["target"])
dep_test=pd.read_csv("D:\\bro\\Analysis\\mSANN\\cd\\XT.dat",sep='\s+',names=[str(i) for i in range(0,16)])
indep_test=pd.read_csv("D:\\bro\\Analysis\\mSANN\\cd\\yT.dat",sep='\s+',names=["target"])

dep_train.shape,indep_train.shape,dep_val.shape,indep_val.shape,dep_test.shape,indep_test.shape

((42039, 16), (42039, 1), (10935, 16), (10935, 1), (10934, 16), (10934, 1))

In [ ]:

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import SGD

def build_model(hyperparams):
    hp_layer_1 = hp.Int('layer_1', min_value=1, max_value=1000, step=50)
    hp_layer_2 = hp.Int('layer_2', min_value=1, max_value=1000, step=50)
    hp_layer_3 = hp.Int('layer_3', min_value=1, max_value=1000, step=50)
    hp_layer_4 = hp.Int('layer_4', min_value=1, max_value=1000, step=50)
    hp_layer_5 = hp.Int('layer_5', min_value=1, max_value=1000, step=50)
    hp_layer_6 = hp.Int('layer_6', min_value=1, max_value=1000, step=50)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    hp_drop_out = hp.Choice('drop_out', values=[0.1, 0.2, 0.15])
    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_weights=['glorot_uniform','he_normal','he_uniform','random_uniform','random_normal']
    model = Sequential()
    model.add(layers.Input(shape=(dep_train.shape[1],)))
    model.add(layers.Dense(units=hp_layer_1, activation=hp_activation,kernel_initializer=hp_weights))
    model.
    model.add(layers.Dense(1))

    optim=hyperparams.Choice("optimizer",["sgd","rmsprop","adam"])
    model.compile(optim, loss="mean_squared_error", metrics=["mean_squared_error"])

    return model